In [1]:
import re
import sys
import os
import math
import csv
from Bio import SwissProt

In [2]:
def is_PTMsite(UniProt_ID,mutation_site):
    indicator=None
    if UniProt_ID in UniProt_PTMsites:
        site = int(re.findall(r'\d+',mutation_site)[0])
        dis = [abs(int(re.findall(r'\d+',elem)[0])-site) for elem in UniProt_PTMsites[UniProt_ID]]
        min_dis = min(dis)
        nearest_site = [elem for elem in UniProt_PTMsites[UniProt_ID] if abs(int(re.findall(r'\d+',elem)[0])-site) == min_dis]
        indicator = [min_dis,nearest_site]
    return indicator


# Acetylation

In [3]:
UniProt_PTMsites={}
with open('Dataset/PTM_dataset/Acetylation merge.csv') as fp:
    PTMreader = csv.reader(fp)
    PTMreader.next()
    for temp in PTMreader:
        UniProt_ID = temp[0]
        if UniProt_ID not in UniProt_PTMsites:
            UniProt_PTMsites[UniProt_ID]=set()
            UniProt_PTMsites[UniProt_ID].add(temp[3])
        else:
            UniProt_PTMsites[UniProt_ID].add(temp[3])

# Malonylation

In [16]:
UniProt_PTMsites={}
with open('Dataset/PTM_dataset/Malonylation-human-merge.csv') as fp:
    PTMreader = csv.reader(fp)
    PTMreader.next()
    for temp in PTMreader:
        UniProt_ID = temp[0]
        if UniProt_ID not in UniProt_PTMsites:
            UniProt_PTMsites[UniProt_ID]=set()
            UniProt_PTMsites[UniProt_ID].add(temp[2])
        else:
            UniProt_PTMsites[UniProt_ID].add(temp[2])

# Methylation

In [19]:
UniProt_PTMsites={}
with open('Dataset/PTM_dataset/Methylation_site_human-merge.csv') as fp:
    PTMreader = csv.reader(fp)
    PTMreader.next()
    for temp in PTMreader:
        UniProt_ID = temp[0]
        if UniProt_ID not in UniProt_PTMsites:
            UniProt_PTMsites[UniProt_ID]=set()
            UniProt_PTMsites[UniProt_ID].add(temp[2])
        else:
            UniProt_PTMsites[UniProt_ID].add(temp[2])

# O-linked glycosylation

In [21]:
UniProt_PTMsites={}
with open('Dataset/PTM_dataset/O-linked glycosylation_human-merge.csv') as fp:
    PTMreader = csv.reader(fp)
    PTMreader.next()
    for temp in PTMreader:
        UniProt_ID = temp[0]
        if UniProt_ID not in UniProt_PTMsites:
            UniProt_PTMsites[UniProt_ID]=set()
            UniProt_PTMsites[UniProt_ID].add(temp[2])
        else:
            UniProt_PTMsites[UniProt_ID].add(temp[2])

# Phosphorylation

In [3]:
UniProt_PTMsites={}
with open('Dataset/PTM_dataset/Phosphorylation_site_human-merge.csv') as fp:
    PTMreader = csv.reader(fp)
    next(PTMreader)
    for temp in PTMreader:
        UniProt_ID = temp[0]
        if UniProt_ID not in UniProt_PTMsites:
            UniProt_PTMsites[UniProt_ID]=set()
            UniProt_PTMsites[UniProt_ID].add(temp[2])
        else:
            UniProt_PTMsites[UniProt_ID].add(temp[2])

In [6]:
PTMsites = open('Dataset/PTMsites.txt','w+')
for ID in UniProt_PTMsites:
    PTMsites.write(ID + '\t' + ','.join(list(UniProt_PTMsites[ID])) + '\n')
PTMsites.close()

# Succinylation

In [26]:
UniProt_PTMsites={}
with open('Dataset/PTM_dataset/Succinylation-human-merge.csv') as fp:
    PTMreader = csv.reader(fp)
    PTMreader.next()
    for temp in PTMreader:
        UniProt_ID = temp[0]
        if UniProt_ID not in UniProt_PTMsites:
            UniProt_PTMsites[UniProt_ID]=set()
            UniProt_PTMsites[UniProt_ID].add(temp[2])
        else:
            UniProt_PTMsites[UniProt_ID].add(temp[2])

# Ubiquitination

In [28]:
UniProt_PTMsites={}
with open('Dataset/PTM_dataset/Ubiquitination_site_human-merge.csv') as fp:
    PTMreader = csv.reader(fp)
    PTMreader.next()
    for temp in PTMreader:
        UniProt_ID = temp[0]
        if UniProt_ID not in UniProt_PTMsites:
            UniProt_PTMsites[UniProt_ID]=set()
            UniProt_PTMsites[UniProt_ID].add(temp[2])
        else:
            UniProt_PTMsites[UniProt_ID].add(temp[2])

# Mutation Mapping

In [5]:
maf_files = [f for f in os.listdir('/Users/junfeizhao/Google Drive/TCGA-Junfei/Somatic Mutation/') if f.endswith('NonSilent.maf')]
for maf in maf_files:
    Cancer_type = maf.split('.')[1]
    fp = open('/Users/junfeizhao/Google Drive/TCGA-Junfei/Somatic Mutation/' + maf)
    next(fp)
    next(fp)
    next(fp)
    Header = next(fp)
    PTMsites_mutations = set()
    #output.write(Header)
    for line in fp:
        temp = line[0:-1].split("\t")
        if temp[8]=='Missense_Mutation':
            gene = temp[0]
            Tumor_ID = temp[15][0:12]
            AA = temp[36]
            AA_pos = re.findall(r'\d+',AA)
            if len(AA_pos) > 0:
                pos = AA_pos[0]
                Protein_ID = temp[68]
                SIFT_score = temp[72]
                PolyPhen_score = temp[73]
                dis = is_PTMsite(Protein_ID,AA[2:-1])
                if dis is not None and dis[0] <= 7:
                    PTMsites_mutations.add('\t'.join([Tumor_ID,gene,Protein_ID,AA,','.join(dis[1]),str(dis[0]),SIFT_score,PolyPhen_score]))
    fp.close()
    output = open('Phosphorylation_mutations/Mutation_mapping/' + Cancer_type + '_Phosphorylation_mutations.txt',"w+")
    output.write('\t'.join(["SampleID","Gene","UniProt","AAchange","Nearest_site","Distance","SIFT_score","PolyPhen_score"]) + '\n')
    output.write('\n'.join(list(PTMsites_mutations)))
    output.close()

In [11]:
UniProt_PTMsites.keys()[0:5]

['Q8TBF4', 'Q5MIZ7', 'Q8TBF2', 'Q8TBF8', 'Q5H9L2']

In [12]:
UniProt_PTMsites['Q8TBF4']

{'K186'}